In [ ]:
import numpy as np
import numpy.random as npr
import torch as th
from torch.autograd import Variable
import torch.nn.functional as F
from torch.optim import Adam
import my

In [ ]:
BATCH_SIZE = 1024
N_CLASSES = 2
N_ITERATIONS = 500
SAMPLE_SIZE = 16
P = (1 / N_CLASSES,) * N_CLASSES
L = lambda y_bar, y: my.nd_f_beta(th.max(y_bar, 1)[1], y, N_CLASSES)
cuda = True
if cuda:
    th.cuda.set_device(3)

In [ ]:
def rand_simplex(n, d):
    x = th.sort(th.rand((n, d - 1)) * 0.001, 1)[0]
    return th.cat((x, th.ones(n, 1)), 1) - th.cat((th.zeros(n, 1), x), 1)

def rand_sample(n, d, p):
    y = Variable(th.from_numpy(npr.choice(np.arange(d), n, True, p)).long())
    y_onehot = my.onehot(y, d)
    y_bar = Variable(rand_simplex(n, d))
    return th.cat((y_onehot, y_bar), 1), L(y_bar, y)

def rand_batch(m, n, d, p):
    x, y = zip(*tuple(rand_sample(n, d, p) for i in range(m)))
    x = tuple(z.view(1, -1) for z in x)
    y = tuple(z.view(1, 1) for z in y)
    return th.cat(x, 0), th.cat(y, 0)

In [ ]:
critic = my.RN(SAMPLE_SIZE, 2 * N_CLASSES, (512,) * 3 + (1,), F.relu)
if cuda:
    critic.cuda()
critic_optim = Adam(critic.parameters(), 1e-3)

In [ ]:
# TODO random sample size
# TODO regularize critic
for i in range(N_ITERATIONS):
    x, y = rand_batch(BATCH_SIZE, SAMPLE_SIZE, N_CLASSES, P)
    if cuda:
        x, y = x.cuda(), y.cuda()
    mse = 1e10
    while float(mse) > 1e-1:
        mse = th.mean((critic(x) - y) ** 2, 0)
        critic_optim.zero_grad()
        mse.backward()
        critic_optim.step()
#     print(float(mse))

    if (i + 1) % 10 == 0:
        x, y = rand_batch(64, SAMPLE_SIZE, N_CLASSES, P)
        if cuda:
            x, y = x.cuda(), y.cuda()
        error = th.mean(th.abs(critic(x) - y), 0)
        print('[iteration %d]error: %f' % (i + 1, error))